In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import sequential
from keras.preprocessing import image

In [ ]:
!unzip "/content/Dataset.rar"

Archive:  /content/Dataset.rar
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/Dataset.rar or
        /content/Dataset.rar.zip, and cannot find /content/Dataset.rar.ZIP, period.


In [ ]:
train_datagen= image.ImageDataGenerator(
    rescale=1/255, horizontal_flip= True, zoom_range=0.2, shear_range=0.2
)

train_data=train_datagen.flow_from_directory(directory="/content/chest_xray_split_png/train", target_size= (256,256), batch_size= 1,
                                             class_mode = 'binary'
                                             )

Found 5154 images belonging to 2 classes.


In [ ]:
train_data.class_indices

{'covid': 0, 'noncovid': 1}

In [ ]:
test_datagen= image.ImageDataGenerator(
    rescale=1/255
)

test_data=test_datagen.flow_from_directory(directory="/content/chest_xray_split_png/test", target_size= (256,256), batch_size= 16,
                                             class_mode = 'binary'
                                             )

Found 1718 images belonging to 2 classes.


In [ ]:
test_data.class_indices

{'covid': 0, 'noncovid': 1}

In [ ]:
from keras.engine.sequential import Sequential
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape= (256,256,3)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25)) 

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25)) 

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.50))
model.add(Dense(units=1, activation='sigmoid'))

model.compile (loss=keras.losses.binary_crossentropy, optimizer="adam", metrics=['acc'])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 conv2d_5 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 126, 126, 64)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 124, 124, 64)      36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                  

In [ ]:
model.fit_generator(train_data, steps_per_epoch=8, epochs=100, validation_steps=2,
                    validation_data=test_data)

In [ ]:
from os import path
path="/content/chest_xray_split_png/train/noncovid/image/1676196.png"
img= image.load_img(path, target_size=(256,256))

img=image.img_to_array(img)/255
img=np.array([img])
img.shape


(1, 256, 256, 3)

In [ ]:
model.predict(img)

In [ ]:
np.argmax(model.predict(img), axis=-1)

array([0])

In [ ]:
(model.predict(img) > 0.5).astype("int32")

array([[1]], dtype=int32)

In [ ]:
Result=model.predict(img)
if model.predict(img)>0.5:
  print ('COVID PATIENT')
else:
  print ('Non-COVID ')


COVID PATIENT
